In [1]:
%pip install langchain-community langchain-pinecone langchain-openai pinecone python-dotenv openpyxl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 21.6 MB/s eta 0:00:00


In [1]:
# 구글 코랩용 코드
from google.colab import userdata
import os


os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
# from dotenv import load_dotenv

# load_dotenv() #.env 파일에서 변수=값을 os환경변수로 자동등록

True

In [3]:
import pandas as pd

df = pd.read_excel('food.xlsx')
df = df[['식품명', '식품중분류명','에너지(kcal)']]
df.columns = ['RCP_NM', 'RCP_PARTS_DTLS', 'INFO_ENG']

df.head()

,RCP_NM,RCP_PARTS_DTLS,INFO_ENG
0,국밥_돼지머리,돼지머리,137
1,국밥_순대국밥,순대국밥,75
2,국밥_콩나물,콩나물,52
3,기장밥,해당없음,166
4,김밥,해당없음,140


In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from tqdm import tqdm

# 임베딩할 텍스트(각각 page_content) 생성 - 메뉴이름, 재료
texts = (df['RCP_NM'] + " / " + df['RCP_PARTS_DTLS']).tolist()

# 메타데이터 생성 - 메뉴이름, 칼로리
metadatas = df[['RCP_NM','INFO_ENG']].to_dict(orient='records')

# OpenAI 임베딩 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Pinecone 인덱스명 지정
index_name = "food-index"

batch_size = 10
max_length = 1000  # 필요시 더 짧게 조정

# 텍스트 길이 제한 적용
texts_limited = [t[:max_length] if len(t) > max_length else t for t in texts]

for i in tqdm(range(0, len(texts_limited), batch_size)):
    batch_texts = texts_limited[i:i+batch_size]
    batch_metadatas = metadatas[i:i+batch_size]
    PineconeVectorStore.from_texts(
        texts=batch_texts,
        embedding=embeddings,
        metadatas=batch_metadatas,
        index_name=index_name
    )

100%|██████████| 1459/1459 [51:23<00:00,  2.11s/it]


In [ ]:
# texts를 벡터화 - 이미지 업로드후 임베딩값과 비교시 사용
text_vectors = embeddings.embed_documents(texts)

1. 이미지 임베딩(벡터화)
- 사용자가 이미지를 업로드하면, 해당 이미지를 이미지 임베딩 모델(CLIP, BLIP)로 벡터값으로 변환. streamlit에서 받은 이미지를 벡터화하는 코드 필요.

2. 위에서 얻은 image_vector를 Pinecone에 쿼리하여, 텍스트 임베딩(레시피 등)과 가장 유사한 결과를 검색.
```results = pinecone_index.query(
    vector=image_vector.tolist(),
    top_k=?,
    include_metadata=True
)```

3. 출력된 결과를 기반으로 llm에 질의 및 응답(과정을 반복하며 식단관리 및 운동을 추천)